# カイ二乗検定のメタアナリシス

## 今回想定しているシチュエーション

「あるパーソナライズされた動線改善を4つの異なるeコマースサイトで行った」<br>
ケースを想定.

4つの eコマースサイトは独立した事業者であるため、統一したアクセス解析はできない<br>
と仮定.<br>
動線改善は「お気に入り→購入」のところに施したとして、ここでA/Bテストを行ったと仮定する.

介入が統計的に有意な差が出るかどうかをRを利用してカイ二乗検定で検証する.

この時、アクセス解析の結果からいわゆる2×2分割表として

#### サイトA

             非CV	CV
改善なし	250	125<br>
改善あり	300	180<br>

#### サイトB

             非CV	CV
改善なし	1500	650<br>
改善あり	1200	620<br>

#### サイトC

             非CV	CV
改善なし	600	300<br>
改善あり	550	310<br>

#### サイトD

            非CV	CV
改善なし	2200	1050<br>
改善あり	1300	700<br>

という集計データが得られたものとする.

積み上げ棒プロットで図示します.

<img src="./data01.png">

## 1. データ作成

まずは行列データとして作成

In [1]:
d1= matrix(c(250,125,300,180),ncol=2,byrow=T)
d2 = matrix(c(1500,650,1200,620),ncol=2,byrow=T)
d3 = matrix(c(600,300,550,310),ncol=2,byrow=T)
d4 = matrix(c(2200,1050,1300,700),ncol=2,byrow=T)

In [2]:
d1

250,125
300,180


In [3]:
d2

1500,650
1200,620


In [4]:
d3

600,300
550,310


In [5]:
d4

2200,1050
1300,700


## 2. カイ二乗検定

一般に2×2分割表で表せるデータに対する効果検証の類は<br>
「独立性の検定」即ちカイ二乗検定で行うことができます.


In [6]:
chisq.test(d1)


	Pearson's Chi-squared test with Yates' continuity correction

data:  d1
X-squared = 1.4164, df = 1, p-value = 0.234


In [7]:
chisq.test(d2)


	Pearson's Chi-squared test with Yates' continuity correction

data:  d2
X-squared = 6.4822, df = 1, p-value = 0.0109


In [8]:
chisq.test(d3)


	Pearson's Chi-squared test with Yates' continuity correction

data:  d3
X-squared = 1.3122, df = 1, p-value = 0.252


In [9]:
chisq.test(d4)


	Pearson's Chi-squared test with Yates' continuity correction

data:  d4
X-squared = 3.9182, df = 1, p-value = 0.04777


### カイ二乗検定の考察

<pre>
d1 p値＝0.234
d２　p値＝0.0109
d３ p値＝0.252
d４　p値＝0.04777
</pre>

有意水準p < 0.05とした場合、<br>
 サイトAは有意でない、Bは有意、Cは有意でない、Dは有意<br>
という結果に.

これでは「それぞれのサイトでパーソナライズされた動線改善が有効だったか否か」<br>
は分かっても、全体として本当に動線改善の効果があったかどうかは何とも言えない.

五分五分で、これでは結論が出せないことになってしまう.


## 3. Cochran-Mantel-Haenszel 検定

異なる時間または場所で繰り返された2×2テーブルのデータがある場合は、<br>
**Cochran-Mantel-Haenszel検定(以下、CMH検定)**を使用します.

前提：独立した2×2のテーブルが複数あること

リピート全体で一定の割合の比率があるかどうかを教えてくれます.

このように2×2の独立性テストで4つの数字にラベルを付ける.
<pre>
     a　　b   
     c  d
</pre>
と（a + b + c + d）= nとすると、Cochran-Mantel-Haenszel検定統計量の式を次のように書くことができます.

$$
\chi_{MH} = \frac{([\sum_i (  a - \frac{ (a + b)(a + c) }{n} ) ] - 0.5)^2 }{ \sum_i ( \frac{(a + b)(a + c)(b + d)(c + d)}{n^3-n^2} )}
$$

分子は、1つのセル（$a$）の観測値と帰無仮説$（a + b）（a + c）/ n$ の期待値との差の絶対値を含むので、<br>
分子は偏差の平方和観測値と期待値の間にある.

**どのように2×2テーブルを配置するかは関係ない**.4つの値のいずれかをaとして使用できます.<br>
連続性補正として0.5を引いています.分母には​​、二乗差の分散の推定値が含まれています.

- 検定統計量$\chi_{MH}$は、観察されたと期待値との差が大きくなるにつれて大きくなる
- 自由度1のカイ二乗分布である。
- CMH検定の式は、異なる出典によって異なる形で示されていますが、それらはすべて代数的に同等.
- ここで示した式には、連続性補正（分子で0.5を引く）が含まれています。これにより、P値がより正確になるはず.


In [10]:
e1<-function(x){
     x[1,1]-(x[1,1]+x[1,2])*(x[1,1]+x[2,1])/sum(x)
 }
e2<-function(x){
     ((x[1,1]+x[1,2])*(x[1,1]+x[2,1])*(x[1,2]+x[2,2])*(x[2,1]+x[2,2]))/(sum(x)^3-sum(x)^2)
}

numerator = ((e1(d1)+e1(d2)+e1(d3)+e1(d4))-0.5)^2
denominator = (e2(d1)+e2(d2)+e2(d3)+e2(d4))
chi2 = numerator / denominator

In [11]:
# Rで計算するには、全確率1からpchisqを引きます。
degree = 1 # 自由度1のカイ二乗分布
1-pchisq(chi2,degree)

[1] 0.0002988727

この例では、有意水準をp < 0.05に置く限りは<br>
「4サイト共通のパーソナライズされた動線改善の効果は有意に見られる」<br>
と言える.

## 4. metafor パッケージによる検定

CRAN<br>
https://cran.r-project.org/<br>
には{metafor}というメタアナリシスに特化したパッケージがあります.

rma.mh関数で、簡単にCMH 検定testの計算ができます.


In [12]:
install.packages('metafor')
library(metafor)

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Loading required package: Matrix
Loading 'metafor' package (version 2.0-0). For an overview 
and introduction to the package please type: help(metafor).


In [13]:
rma.mh(c(d1[1,1],d2[1,1],d3[1,1],d4[1,1]),
       c(d1[1,2],d2[1,2],d3[1,2],d4[1,2]),
       c(d1[2,1],d2[2,1],d3[2,1],d4[2,1]),
       c(d1[2,2],d2[2,2],d3[2,2],d4[2,2]))


Fixed-Effects Model (k = 4)

Test for Heterogeneity: 
Q(df = 3) = 0.4987, p-val = 0.9192

Model Results (log scale):

estimate      se    zval    pval   ci.lb   ci.ub
  0.1437  0.0395  3.6361  0.0003  0.0662  0.2212

Model Results (OR scale):

estimate   ci.lb   ci.ub
  1.1546  1.0685  1.2476

Cochran-Mantel-Haenszel Test:    CMH = 13.0774, df = 1, p-val = 0.0003
Tarone's Test for Heterogeneity: X^2 =  0.4987, df = 3, p-val = 0.9192


### rma.mh 実行結果の考察
Cochran-Mantel-Haenszel Test:の欄を確認すると、

CMH = 13.0774, <br>
df(degree of freedom:自由度) = 1<br>
p-val = 0.0003< 0.05 となり「効果は有意に見られる」ことを確認できる

そもそもこのようなメタアナリシスを行って良いかどうかの指標にもなる<br>
heterogeneityのチェックもできます.以下ではこのheterogeneityについて見ていきます.

## 5. 異質性 (heterogeneity)

異質性(heterogeneity)とは、簡単に言えばメタ解析の結果のバラつき具合のこと

<pre>
Test for Heterogeneity: 
Q(df = 3) = 0.4987, p-val = 0.9192
</pre>

コクランのQ検定はp値が小さいほど異質性が高いとみなされます.


## 参考文献

Cochran–Mantel–Haenszel test for repeated tests of independence<br>
http://www.biostathandbook.com/cmh.html

Rで実践する統計的検定の初歩<br>
http://www.atmarkit.co.jp/ait/articles/1008/04/news090_2.html

CRAN - The Comprehensive R Archive Network<br>
https://cran.r-project.org/

メタ解析の異質性(heterogeneity)について知る　<br>
https://ameblo.jp/intelligent-pharmacist/entry-12123408452.html

メタ・アナリシスの入門<br>
https://www.slideshare.net/YoshihikoKunisato/ss-37490113

明日から読めるメタ・アナリシス<br>
https://www.slideshare.net/okumurayasuyuki/meta-analysis-okumura